# Project 3 - Nanjing Navigator 3000

By Jay Irby, Ben Lang, and David Zapata

Importing packages - do not import anything that we will not use.

In [38]:
import numpy as np
import csv
# from google.colab import files ** For previous file manipulation purposes
import urllib.request
import heapq
import time
import folium
from IPython.display import display, clear_output
import ipywidgets as widgets

Installations for visualization component

In [39]:
!pip install ipywidgets

In [40]:
!pip install folium

Our first step:

After downloading, zipping, and importing data into the file, need to convert to csv.

IMPORTANT NOTE: Just for our use, and we ran it, it worked, we put the .csv's on our repo, and now it's commented out.

Ben wrote this code - was responsible for creating the Nanjing graph into an adjacency list

In [36]:
# Converting to csv:
# uploaded = files.upload()
# input_name = next(iter(uploaded))
# output_name = "Nanjing_distances.csv"

# with open(input_name, 'r') as infile, open(output_name, 'w', newline='') as outfile:
#   writer = csv.writer(outfile)
#   writer.writerow(["Source", "Destination", "Weight"])

#   for line in infile:
#     line = line.strip()
#     if not line or line.startswith('#'):
#       continue

#     parts = line.split()
#     if len(parts) < 3:
#       print(f"Skipping line (not enough parts): {line}")
#       continue

#     source, destination, weight = parts[0], parts[1], parts[2]
#     writer.writerow([source, destination, weight])

Jay wrote this code - was responsible for turning the .co coordinates file into a map with each node and their corresponding lat, lon coords

In [37]:
# # Converting .co to csv
# uploaded = files.upload()
# input_name = next(iter(uploaded))
# output_name = "coordinates.csv"

# count = 0
# with open(input_name, 'r') as infile, open(output_name, 'w', newline='') as outfile:
#     # Skip the first 6 lines
#   for i in range(6):
#     infile.readline()

#   writer = csv.writer(outfile)
#   writer.writerow(["Source", "X", "Y"])

#   for line in infile:
#     line = line.strip()
#     if not line or line.startswith('#'):
#         continue

#     parts = line.split()
#     if len(parts) < 2:
#         print(f"Skipping line (not enough parts): {line}")
#         continue

#     source = count
#     x = parts[0]
#     y = parts[1]

#     writer.writerow([source, x, y])
#     count += 1

# Functions for converting .csv's into our desired data structure

Like said before, Ben wrote this

In [41]:
# Read in file
def read_graph(file_path):
  """
  Reads a graph from a file and returns it as an adjacency list.

  Args:
    file_path: The path to the file containing the graph.

  Returns:
    A graph representing the adjacency list of the graph.
  """
  graph = {}
  with open(file_path, 'r', newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
      try:
        u = int(row["Source"])
        v = int(row["Destination"])
        weight = float(row["Weight"])
      except ValueError as e:
        print(f"Skipping row with {row} due to error: {e}")
        continue

      if u not in graph:
        graph[u] = []
      graph[u].append((v, weight))

  return graph

Like said before, Jay wrote this

In [42]:
def read_coords(file):
  """
  Reads a coordinate df and returns it as a paired map: {source: (lon, lad)}
  Sources are represented as ints

  Args:
    file: The path to the file containing the graph.

  Returns:
    A map representing the source and their respective coordinates
  """


  map = {}

  with open(file, 'r', newline='') as csvfile:
    reader = csv.DictReader(csvfile)

    for row in reader:
      try:
        source = int(row["Source"])
        lad = float(row["X"])
        lon = float(row["Y"])
      except ValueError as e:
        print(f"Skipping row with {row} due to error: {e}")
        continue

      map[source] = (lad, lon)

  return map

# Importing distances graph and coordinates file using our functions

Jay did all of this

Distances Graph:

In [43]:
# For reproducibility:
url = "https://raw.githubusercontent.com/jayirby2/DSA-Project-3---Nanjing-Navigator-3000-/refs/heads/main/Nanjing_distances.csv"
urllib.request.urlretrieve(url, "Nanjing_distances.csv")

('Nanjing_distances.csv', <http.client.HTTPMessage at 0x7895c4ed0290>)

In [44]:
# Read csv file using our function:

nanjing_graph = read_graph("Nanjing_distances.csv")

Coordinates map:

In [45]:
url_coords = "https://raw.githubusercontent.com/jayirby2/DSA-Project-3---Nanjing-Navigator-3000-/refs/heads/main/coordinates.csv"
urllib.request.urlretrieve(url_coords, "Nanjing_coords.csv")

('Nanjing_coords.csv', <http.client.HTTPMessage at 0x7895c5a8db50>)

In [46]:
# Read csv file using our function:

nanjing_coords = read_coords("Nanjing_coords.csv")

# Helper Functions for Algorithms

In [47]:
# David wrote this function
def calc_distances(path):
  """
  Calculates total distances of the path
  args:
    path: the path to the location returned by algorithm function

  returns:
    total distance float
  """

  # Iterate through path list, adding together the weights of all edges
  total_distance = 0
  for i in range(len(path) - 1):
    for j in range(len(nanjing_graph[path[i]])):
      if nanjing_graph[path[i]][j][0] == path[i+1]:
        total_distance += nanjing_graph[path[i]][j][1]

  return total_distance


# Ben wrote this function
def reconstruct_path(previous, start, target):
  """
  Reconstructs the shortest path from the target node to the start node.
  Args:
    previous: A dictionary mapping each node to its predecessor node in the shortest path.
    start: The start node of the shortest path.
    target: The target node of the shortest path.
  Returns:
    list: A list of nodes representing the shortest path from the start node to the target node.
  """
  path = []
  node = target
  while node is not None:
    path.append(node)
    if node == start:
      break
    node = previous[node]
  path.reverse()
  if path[0] == start:
    return path
  return []



# Euclidean Distances function for calculating raw distances between locations in A*
# Jay wrote this function

def euc_dist(node1, node2, coords=nanjing_coords):
  """
  Returns the euclidean distance between 2 locations.
  Params:
    node1: int name of location
    node2: int name of other location

  Returns:
    Euclidean distance between these 2 locations.
  """

  lat1, lon1 = coords[node1]
  lat2, lon2 = coords[node2]

  # Euclidean distance formula between two points in space
  return ((lat2-lat1)**2 + (lon2-lon1)**2)**0.5 # Tried using numpy at first, but was somehow slower?? Maybe a google collab issue...


# David wrote this function
def visualize_path(path, color="purple", coords=nanjing_coords):
  """
  Returns a visual map powered by OpenStreetMap with starting point, destination,
  and a colored path that connects the two

  Params:
    path: shortest path as a list (returned from either function)
    color: color of the path (default purple)
    coords: coordinates map, default is our nanjing coordinates maps (lat, lon)

  Returns:
    map: A visual map powered by OpenStreetMap with starting point, destination,
    and a colored path that connects the two
  """

  # Set some lat and lon so folium knows where to create a zoomed in map
  lat, lon = coords[path[0]]

  # Use folium to create the map and using our lat, lon as the location
  map = folium.Map(location=[lat, lon], zoom_start=10)

  # Draw the path node by node in the list
  for i in range(len(path) - 1):
    lat1, lon1 = coords[path[i]]
    lat2, lon2 = coords[path[i + 1]]

    # Draw each line
    folium.PolyLine(locations=[[lat1, lon1], [lat2, lon2]], color=color, weight=4).add_to(map)

  # Add visual pointers for start and destination
  start_lat, start_lon = coords[path[0]]
  destination_lat, destination_lon = coords[path[-1]]
  folium.Marker([start_lat, start_lon], popup="Start", icon=folium.Icon(color='blue')).add_to(map)
  folium.Marker([destination_lat, destination_lon], popup="Destination", icon=folium.Icon(color='green')).add_to(map)

  return map


# Dijkstra's Algorithm Implementation

Now that we have the data, let's implement Dijkstra's algorithm on Nanjing's road map, using distances as the weights.

Note: Typically, Dijkstra's algorithm computes shortest path from inserted node to every other node in the graph. Here, we just stop at the desired destination for efficiency. Also, it is more fair when comparing to A*, because A* also technically stops at desired node.

Ben wrote this function - was responsible for Dijkstra's algo implementation

In [48]:
def dijkstra(start, destination, graph=nanjing_graph):
  """
  Performs Dijkstra's algorithm on a graph starting from a given node.
  Arguments:
    graph: A graph represented as an adjacency list; set to our nanjing_graph.
    start: The node to start the algorithm from.
    destination: Ending location for shortest path we want to construct
  Returns:
    Shortest path from start to destination in form of a list, node by node
  """
  # Initialize starting dictionaries (maps) for distances and previous node
  distances = {node : float('inf') for node in graph}
  previous = {node : None for node in graph}
  distances[start] = 0

  # Initialize heap and set for performing the algorithm
  heap =[(0, start)]
  visited = set()
  count = 0

  # Iterate through heap visiting neighbors
  while heap:
    curr_distance, curr_node =  heapq.heappop(heap)
    visited.add(curr_node)
    count += 1

    # Checks if reached destination
    if curr_node == destination:
      break

    # Checks if distance to node should be updated or not
    if curr_distance > distances[curr_node]:
      continue

    # Iterates through neighbors checking for shortest distance and updating previous node as needed
    for neighbor, weight in graph[curr_node]:
      distance = curr_distance + weight
      if distance < distances[neighbor]:
        distances[neighbor] = distance
        previous[neighbor] = curr_node
        heapq.heappush(heap, (distance, neighbor))

  return reconstruct_path(previous, start, curr_node)

# A* Algorithm Implementation

We will be using both the coordinates map and distances graph

Jay wrote this function - was responsible for A* algo implementation

In [49]:
def a_star(start, destination, graph=nanjing_graph, coords=nanjing_coords):
  """
  Calculates the shortest path between two nodes

  Params:
    start: starting location
    destination: desired location
    graph: our nanjing street map graph
    coords: coordinates of the locations (latitude, longitude)

  Returns:
    The path of shortest distance (list of nodes from start to end)
  """

  # Cost is the shortest distance from start node to a particular node 'n'
  # Initialize dictionaries (maps)
  cost = {node : float('inf') for node in graph}
  cost[start] = 0

  # Track previous nodes for resulting shortest path list reconstruction
  previous = {}

  # Use a min heap for extracting node with lowest A* score
  heap = [(euc_dist(start, destination), start)]

  # For testing purposes
  visited = set()
  count = 0

  while heap:

    # Retrieve node with lowest A* score using a min heap
    current_score, curr_node = heapq.heappop(heap)
    visited.add(curr_node)
    count += 1

    # If destination is reached, backtrack to get path and return
    if curr_node == destination:
      path = [curr_node]
      while curr_node in previous:
          curr_node = previous[curr_node]
          path.append(curr_node)
      return path[::-1]

    # If it is not target node, then check neighboring nodes (adjacent to current in adjacency list)
    for neighbor, distance in graph[curr_node]:

      # Increment how much distance has been traveled
      particular_cost = cost[curr_node] + distance

      # If path to neighbor is shorter
      if particular_cost < cost[neighbor]:

        # Add to the running path
        previous[neighbor] = curr_node

        # Set how much distance has been traveled so far to the neighbor's cost
        cost[neighbor] = particular_cost

        # Calculate the score for the neighbor
        score_neighbor = particular_cost + euc_dist(neighbor, destination)

        # Push the score and neighbor node to the heap
        heapq.heappush(heap, (score_neighbor, neighbor))

# Visualizing and Comparing The Two Algorithms

Click 'Compare Algos' button to run the simulation as many times as you want. **Note: in order to start simulation, click 'compare algos' button**

David wrote this code and function - was responsible for visualization

In [50]:
button = widgets.Button(description="Compare Algos") # Create the button
output = widgets.Output()

def on_button(b):
  with output:
    clear_output(wait=True)

    # Random start and destination
    nodes = list(nanjing_graph.keys())
    start = int(np.random.choice(nodes))
    destination = int(np.random.choice(nodes))

    # Edge case for if start and destination locations are generated to be the same
    while destination == start:
        destination = int(np.random.choice(nodes))

    print(f"Start: {start} || Destination: {destination}\n")

    # Dijkstra
    time_start = time.perf_counter()
    d_path = dijkstra(start, destination)
    time_end = time.perf_counter()
    d_time = time_end - time_start
    d_dist = calc_distances(d_path)

    # A*
    time_start = time.perf_counter()
    a_path = a_star(start, destination)
    time_end = time.perf_counter()
    a_time = time_end - time_start
    a_dist = calc_distances(a_path)

    # Printing some statistics
    print(f"Dijkstra's distance: {d_dist:.2f}m || Time: {d_time:.3f}s")
    print(f"A* distance: {a_dist:.2f}m || Time: {a_time:.3f}s")
    if d_path == a_path:
        print("Same path\n")
    elif d_dist == a_dist:
        print("Same distance but different path\n")
    else:
        print("Different distance and/or path\n")

    # Side by side map outputs (paths are hopefully gonna be equal, but in case they aren't for some reason)
    D_map = widgets.Output()
    A_map = widgets.Output()
    print("Dijkstra's path (on left)                         A* path (on right)")

    with D_map:
        display(visualize_path(d_path))

    with A_map:
        display(visualize_path(a_path, color='blue'))

    display(widgets.HBox([D_map, A_map]))



# Button
button.on_click(on_button)
display(button, output)

Button(description='Compare Algos', style=ButtonStyle())

Output()